# Dataset Downloads

This notebooks handles the download of all datasets used in this study.


In [1]:
import os
import zipfile

import cdsapi
import pandas as pd
import xarray as xr
from tqdm.std import tqdm

from deeprec.utils import ROOT_DIR, download_file, download_zip

# Set download path
DL_PATH = ROOT_DIR / "data/raw"
DL_PATH.mkdir(parents=True, exist_ok=True)
print(f"Download path: {DL_PATH}")

Download path: C:\Users\luisg\Repositories\deeprec\data\raw


## Mascons

### JPL Mascons

Global surface mass changes (land + ocean) updated monthly, and is provided on 0.5-degree global grids ([Dataset description](https://grace.jpl.nasa.gov/data/get-data/jpl_global_mascons/)).

In [6]:
start_date = "2002-04-04"
end_date = "2024-12-31"

dataset = "TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.3_V4"
outdir = DL_PATH / "targets/jpl-mascons"

In [3]:
def to_podaac_datetime(date: str) -> str:
    return pd.to_datetime(date).strftime("%Y-%m-%dT%H:%M:%SZ")

In [7]:
!(podaac-data-downloader -c { dataset } -d { outdir } -sd { to_podaac_datetime(start_date) } -ed { to_podaac_datetime(end_date) } -e ".nc")

[2024-12-05 11:42:35,128] {podaac_data_downloader.py:317} INFO - Found 1 total files to download
[2024-12-05 11:42:43,625] {podaac_data_downloader.py:358} INFO - 2024-12-05 11:42:43.625576 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.3_V4/GRCTellus.JPL.200204_202409.GLO.RL06.3M.MSCNv04CRI.nc
[2024-12-05 11:42:43,625] {podaac_data_downloader.py:369} INFO - Downloaded Files: 1
[2024-12-05 11:42:43,625] {podaac_data_downloader.py:370} INFO - Failed Files:     0
[2024-12-05 11:42:43,625] {podaac_data_downloader.py:371} INFO - Skipped Files:    0
[2024-12-05 11:42:43,625] {podaac_data_downloader.py:177} INFO - Success Count: 1
[2024-12-05 11:42:44,027] {podaac_data_downloader.py:186} INFO - END




### CSR Mascons

[Website](https://www2.csr.utexas.edu/grace/RL06_mascons.html)

In [ ]:
outdir = DL_PATH / "targets/csr-mascons"
url = "https://download.csr.utexas.edu/outgoing/grace/RL0603_mascons/CSR_GRACE_GRACE-FO_RL0603_Mascons_all-corrections.nc"
download_file(url, outdir)

### GSFC Mascons

[Website](https://earth.gsfc.nasa.gov/geo/data/grace-mascons)


In [4]:
outdir = DL_PATH / "targets/gsfc-mascons"
url = "https://earth.gsfc.nasa.gov/sites/default/files/geo/gsfc.glb_.200204_202406_rl06v2.0_obp-ice6gd_halfdegree.nc"
download_file(url, outdir)

## Hydrologic models

### WaterGAP Global Hydrology Model (WGHM) 2.2e

- [Dataset on the Goethe University Data Repository](https://gude.uni-frankfurt.de/entities/researchdata/c53bb505-a620-4860-b2a2-d5a6de74dbd9/details)
- [Preprint of paper on WaterGAP 2.2e](https://doi.org/10.5194/gmd-2023-213)

*Download links might change in the future, currently there is no public API available.*

In [4]:
outdir = DL_PATH / "inputs/watergap22e"
urls = {
    "20crv-era5": "https://api.gude.uni-frankfurt.de/api/core/bitstreams/07183cd6-9d47-4cb2-bc60-00436b0ecd39/content",
    "gswp3-era5": "https://api.gude.uni-frankfurt.de/api/core/bitstreams/879ce7c3-4d21-4ee1-a83c-e830b13b9d2e/content",
    "20crv-w5e5": "https://api.gude.uni-frankfurt.de/api/core/bitstreams/2b22924a-0981-4f6c-886c-542d19db7783/content",
    "gswp3-w5e5": "https://api.gude.uni-frankfurt.de/api/core/bitstreams/adee0d04-c414-420e-85ef-89d3e83e32e9/content",
}
for name, url in (pbar := tqdm(urls.items())):
    pbar.set_postfix_str(f"Downloading watergap22e_{name}")
    download_file(url, outdir)

100%|██████████| 4/4 [00:12<00:00,  3.12s/it, Downloading watergap22e_gswp3-w5e5]


## Weather and climate data

### ERA5

ERA5 data could be combined with the ISIMIP 20CRv3-ERA5 dataset to make up the missing years 2022 and 2023.

- Temporal coverage: 1940 to present
- Temporal frequency:
  - hourly
  - monthly averaged by hour of day (synoptic monthly means)
  - monthly averaged
- Spatial resolution:
  - Reanalysis: 0.25° x 0.25° (atmosphere), 0.5° x 0.5° (ocean waves)
  - Mean, spread and members: 0.5° x 0.5° (atmosphere), 1° x 1° (ocean waves)


Useful links:
- [CDS: ERA5 single levels monthly means](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means)
- [ERA5 data documentation](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation)
- [How to download ERA5](https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5)

In [2]:
START_YEAR = 1940
END_YEAR = 2023
VARIABLES = [
    "total_precipitation",
    "2m_temperature",
    "2m_dewpoint_temperature",
    "high_vegetation_cover",
    "low_vegetation_cover",
    "evaporation",
    "potential_evaporation",
    "runoff",
    "snowfall",
    "snowmelt",
    "snow_depth",
    "snow_evaporation",
    "surface_pressure",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation",
    "sub_surface_runoff",
    "surface_runoff",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4",
]

dataset_path = DL_PATH / "inputs/era5-monthly"
dataset_path.mkdir(exist_ok=True)

c = cdsapi.Client()
dataset = "reanalysis-era5-single-levels-monthly-means"
for variable in VARIABLES:
    print(f"{'=' * 40}")
    print(f"Downloading `{variable}`...")

    request = {
        "product_type": ["monthly_averaged_reanalysis"],
        "variable": [variable],
        "year": [f"{year}" for year in range(START_YEAR, END_YEAR + 1)],
        "month": [f"{month:02}" for month in range(1, 12 + 1)],
        "time": ["00:00"],
        "data_format": "netcdf",
        "download_format": "unarchived",
    }
    dataset_file = dataset_path / f"era5-monthly_{variable}_{START_YEAR}-{END_YEAR}.nc"
    c.retrieve(dataset, request, dataset_file)

print("Download completed.")

2024-12-05 13:23:41,111 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-05 13:23:41,112 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-12-05 13:23:41,112 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-12-05 13:23:41,113 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

2024-12-05 13:23:44,626 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-05 13:23:44,627 INFO Request ID is f955ee6e-2e61-48a4-8932-e385d8ab13a2
2024-12-05 13:23:44,822 INFO status has been updated to accepted
2024-12-05 13:24:01,235 INFO status has been updated to successful


458d2625383eebb0496ef65fb8522080.nc:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

2024-12-05 13:28:11,818 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-05 13:28:11,819 INFO Request ID is fa7b5091-71b6-45d5-83a1-12852ad95f5d
2024-12-05 13:28:13,183 INFO status has been updated to accepted
2024-12-05 13:28:21,626 INFO status has been updated to running
2024-12-05 13:30:07,425 WARNING Structural differences in grib fields detected when opening in xarray. Opening the grib file safely, however this may result in files with non-intuitive filenames.
2024-12-05 13:32:32,196 INFO status has been updated to successful


21a9f27481bfba6b8cb809a0bc2209d8.nc:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

...

2024-12-05 15:20:05,025 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-05 15:20:05,026 INFO Request ID is 1e7e38ba-d6b6-4eb7-ba60-af8fd88063f2
2024-12-05 15:20:05,285 INFO status has been updated to accepted
2024-12-05 15:20:12,406 INFO status has been updated to running
2024-12-05 15:22:03,179 WARNING Structural differences in grib fields detected when opening in xarray. Opening the grib file safely, however this may result in files with non-intuitive filenames.
2024-12-05 15:24:29,195 INFO status has been updated to successful


3a098a403b8fb8b23bc852bb2e0ec1cd.nc:   0%|          | 0.00/707M [00:00<?, ?B/s]

2024-12-05 15:26:25,132 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-05 15:26:25,132 INFO Request ID is d1e39218-b346-4241-8960-700c1962c1e3
2024-12-05 15:26:25,234 INFO status has been updated to accepted
2024-12-05 15:26:30,230 INFO status has been updated to running
2024-12-05 15:28:20,010 WARNING Structural differences in grib fields detected when opening in xarray. Opening the grib file safely, however this may result in files with non-intuitive filenames.
2024-12-05 15:29:17,876 INFO status has been updated to successful


b900c1c6aa64cd4229ed330da3fdfc2c.nc:   0%|          | 0.00/449M [00:00<?, ?B/s]

Download completed.


### NOAA Reconstructed Sea Surface Temperature

Download the sea surface temperatures (SST) for calculating the Ocenanic Nino index (ONI). The ONI climate indices provided by [NOAA](https://psl.noaa.gov/data/climateindices/list/) only span to 1950.

[Dataset description](https://psl.noaa.gov/data/gridded/data.noaa.ersst.v5.html)

In [2]:
outdir = DL_PATH / "inputs/noaa-ersst-v5"
url = "https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc"
download_file(url, outdir)

### Glacier mass change

[Dataset description](https://cds.climate.copernicus.eu/datasets/derived-gridded-glacier-mass-change)

In [8]:
START_YEAR = 1975
END_YEAR = 2021

dataset_path = DL_PATH / "inputs/wgms-fog"
dataset_path.mkdir(exist_ok=True)
dataset_file = dataset_path / "wgms_fog_2023_09.zip"

c = cdsapi.Client()
dataset = "derived-gridded-glacier-mass-change"
request = {
    "variable": "glacier_mass_change",
    "hydrological_year": [
        f"{year}_{(year + 1) % 100:02}" for year in range(START_YEAR, END_YEAR + 1)
    ],
    "product_version": "wgms_fog_2023_09",
}
c.retrieve(dataset, request, dataset_file)
print("Download completed.")

2024-12-05 11:01:41,093 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-05 11:01:41,093 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-12-05 11:01:41,094 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-12-05 11:01:41,094 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

617ff9b5a8163916054bb6132545eced.zip:   0%|          | 0.00/467M [00:00<?, ?B/s]

Download completed.


In [ ]:
# Extract ZIP
with zipfile.ZipFile(dataset_file) as zip_file:
    zip_file.extractall(dataset_path)
# Remove ZIP
dataset_file.unlink()

# Combine yearly files
yearly_files = list(dataset_path.glob("*.nc4"))
with xr.open_mfdataset(yearly_files) as ds:
    ds.to_netcdf(dataset_path / "wgms-fog-2023-09_glacier-mass-change.nc")
# Remove yearly files
for file in yearly_files:
    file.unlink()

## Human influences

### ISIMIP Land Use

[Data set description](https://www.isimip.org/gettingstarted/input-data-bias-adjustment/details/82/)

In [4]:
outdir = DL_PATH / "inputs/landuse"
urls = {
    "5crops": "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/landuse/histsoc/landuse-5crops_histsoc_annual_1901_2021.nc",
    "15crops": "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/landuse/histsoc/landuse-15crops_histsoc_annual_1901_2021.nc",
    "pastures": "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/landuse/histsoc/landuse-pastures_histsoc_annual_1901_2021.nc",
    "totals": "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/landuse/histsoc/landuse-totals_histsoc_annual_1901_2021.nc",
    "urbanareas": "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/landuse/histsoc/landuse-urbanareas_histsoc_annual_1901_2021.nc",
}
for name, url in (pbar := tqdm(urls.items(), desc="Downloading landuse")):
    pbar.set_postfix_str(name)
    download_file(url, outdir)

### ISIMIP Lake area fraction

[Data set description](https://www.isimip.org/gettingstarted/input-data-bias-adjustment/details/132/)

In [3]:
outdir = DL_PATH / "inputs/pctlake"
url = "https://files.isimip.org/ISIMIP3a/InputData/socioeconomic/lakes/histsoc/pctlake_histsoc_1901_2021.nc"
download_file(url, outdir)

## Shapes

### GRDC Major River Basins

Major river basins by the Global Runoff Data Center. The basins incorporate HydroBASINS data and are named. They are available as Shapefile and geoJSON

- [Description](https://www.bafg.de/GRDC/EN/02_srvcs/22_gslrs/221_MRB/riverbasins_node.html)
- [Map and download](https://mrb.grdc.bafg.de/)

In [ ]:
url = "https://grdc.bafg.de/downloads/GRDC_Major_River_Basins_shp.zip"
path = DL_PATH / "shapefiles/mrb"

download_zip(url, path)
print("Download completed.")

Download completed.


### NaturalEarth 1:50 countries

[Description](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/50m-admin-0-countries-2/)

In [ ]:
url = "https://naturalearth.s3.amazonaws.com/5.0.1/50m_cultural/ne_50m_admin_0_countries.zip"
path = DL_PATH / "shapefiles/naturalearth"

download_zip(url, path)
print("Download completed.")

Download completed.


## Previous TWS reconstructions
### Humphrey, 2019

[Data on figshare](https://figshare.com/articles/dataset/GRACE-REC_A_reconstruction_of_climate-driven_water_storage_changes_over_the_last_century/7670849)

In [16]:
url = "https://figshare.com/ndownloader/files/17990285"
path = DL_PATH / "reconstructions/humphrey"
path.mkdir(parents=True, exist_ok=True)

download_zip(url, path)
print("Download completed.")

Download completed.


### Li, 2021

[Download here on DRYAD](https://datadryad.org/stash/dataset/doi:10.5061/dryad.z612jm6bt)

### Chandanpurkar, 2022

[Data on zenodo](https://zenodo.org/records/6659543)

In [15]:
url = "https://zenodo.org/records/6659543/files/tws_cseof_v1.nc"
path = DL_PATH / "reconstructions/chandanpurkar"

download_file(url, path)
print("Download completed.")


Download completed.


### Yin, 2023

[Data on zenodo](https://zenodo.org/records/10040927)

In [ ]:
path = DL_PATH / "reconstructions/yin"
urls = [
    "https://zenodo.org/records/10040927/files/CSR-based%20GTWS-MLrec%20TWS.nc",
    "https://zenodo.org/records/10040927/files/GSFC-based%20GTWS-MLrec%20TWS.nc",
    "https://zenodo.org/records/10040927/files/JPL-based%20GTWS-MLrec%20TWS.nc",
]

for url in tqdm(urls):
    download_file(url, path)


### Palazzoli, 2025

[Data on zenodo](https://zenodo.org/records/10953658)

In [ ]:
url = "https://zenodo.org/records/10953658/files/GRAiCE_BiLSTM.nc"
path = DL_PATH / "reconstructions/palazzoli"

download_file(url, path)
print("Download completed.")
